In [18]:
'''
date: 2023/2/19
author: LIU Guangyao
description: generate affinity matrix using pixels

'''
import numpy as np
from skimage.color import rgb2lab
from scipy.spatial.distance import cdist
from scipy.sparse import csr_matrix
import cv2
import torch

np.set_printoptions(threshold=np.inf)

def calculate_texture_feat(img):
    # Convert the image to LAB color space
    lab = rgb2lab(img)

    # Compute the squared Euclidean distance between color values of neighboring pixels
    texture_feat = np.zeros((lab.shape[0], lab.shape[1], 3))
    for i in range(1, lab.shape[0] - 1):
        for j in range(1, lab.shape[1] - 1):
            texture_feat[i, j, 0] = np.sum((lab[i, j, 0] - lab[i-1:i+2, j-1:j+2, 0].flatten())**2)
            texture_feat[i, j, 1] = np.sum((lab[i, j, 1] - lab[i-1:i+2, j-1:j+2, 1].flatten())**2)
            texture_feat[i, j, 2] = np.sum((lab[i, j, 2] - lab[i-1:i+2, j-1:j+2, 2].flatten())**2)

    return texture_feat

def generate_affinity_matrix(img, alpha=0.1, wc=1, ws=1, wt=1, sc=10, ss=10, st=10):
    # Convert the image to LAB color space
    lab = rgb2lab(img)

    # Compute the color, spatial, and texture features
    color_feat = lab.reshape((-1, 3))
    index = color_feat.shape[0]
    print(index)
    spatial_feat = np.column_stack(np.meshgrid(np.arange(img.shape[0]), np.arange(img.shape[1]))).reshape((-1, 2))
    # texture_feat = np.zeros((index*index, 3), dtype=float)
    # print(texture_feat.shape)
    print('spatial feat size:',spatial_feat.shape)
    print('color feat size:',color_feat.shape)
    '''
    texture_feat[:, 0] = cdist(lab[:, :, 0].reshape((-1, 1)), lab[:, :, 0].reshape((-1, 1)), metric='sqeuclidean').reshape((-1,))
    texture_feat[:, 1] = cdist(lab[:, :, 1].reshape((-1, 1)), lab[:, :, 1].reshape((-1, 1)), metric='sqeuclidean').reshape((-1,))
    texture_feat[:, 2] = cdist(lab[:, :, 2].reshape((-1, 1)), lab[:, :, 2].reshape((-1, 1)), metric='sqeuclidean').reshape((-1,))
    '''

    # Compute the pairwise distances between features
    color_dist = cdist(color_feat, color_feat, metric='sqeuclidean')
    spatial_dist = cdist(spatial_feat, spatial_feat, metric='euclidean')
    # texture_dist = cdist(texture_affinity, texture_affinity, metric='sqeuclidean')

    # Compute the affinity matrix
    affinity = np.exp(-alpha * (wc * color_dist / sc**2 + ws * spatial_dist / ss**2 ))

    # Convert the affinity matrix to a sparse matrix
    sparse_affinity = csr_matrix(affinity)
    print('sparse_affinity.shape', affinity.shape)
    # print(affinity)
    
    # tensor = torch.tensor(affinity)
    # file_path = './affinity/lina2.txt'
    # torch.save(tensor, file_path)
    np.savetxt('./affinity/lina2.txt', affinity)
    # np.savetxt('./affinity/lina.txt', sparse_affinity)
    return sparse_affinity

def update(input_img_path, output_img_path):
    image = cv2.imread(input_img_path)
    print('original size: ',image.shape)
    cropped = image[0:4, 0:4] # 裁剪坐标为[y0:y1, x0:x1]
    cv2.imwrite(output_img_path, cropped)
    print('cropped size: ',cropped.shape)

input = './img/Lena.jpg'
output = 'imgout/new.jpg'
update(input, output)
img = cv2.imread('./imgout/new.jpg')
generate_affinity_matrix(img)

original size:  (32, 32, 3)
cropped size:  (4, 4, 3)
16
spatial feat size: (16, 2)
color feat size: (16, 3)
sparse_affinity.shape (16, 16)


<16x16 sparse matrix of type '<class 'numpy.float64'>'
	with 256 stored elements in Compressed Sparse Row format>

In [19]:
import numpy as np
# np.loadtxt('./affinity/lina2.txt')

In [8]:
'''
date: 2023/2/19
author: LIU Guangyao
description: generate affinity matrix using superpixels

'''
import numpy as np
from skimage import io
from skimage.color import rgb2lab
from skimage.segmentation import slic
from scipy.spatial.distance import cdist
from scipy.sparse import csr_matrix

def generate_affinity_matrix(img, num_segments, alpha=0.1, wc=1, ws=1, wt=1, sc=10, ss=10, st=10):
    # Segment the image into superpixels using SLIC
    segments = slic(img, n_segments=num_segments, compactness=10, sigma=1)

    # Convert the image to LAB color space
    lab = rgb2lab(img)

    # Compute the color, spatial, and texture features for each superpixel
    color_feat = np.zeros((num_segments, 3))
    spatial_feat = np.zeros((num_segments, 2))
    texture_feat = np.zeros((num_segments, 3))
    for i in range(num_segments):
        mask = segments == i
        color_feat[i] = np.mean(lab[mask], axis=0)
        spatial_feat[i] = np.mean(np.argwhere(mask), axis=0)
        texture_feat[i, 0] = np.mean(cdist(lab[mask, 0].reshape((-1, 1)), lab[segments == (i - 1), 0].reshape((-1, 1)), metric='sqeuclidean'))
        texture_feat[i, 1] = np.mean(cdist(lab[mask, 1].reshape((-1, 1)), lab[segments == (i - 1), 1].reshape((-1, 1)), metric='sqeuclidean'))
        texture_feat[i, 2] = np.mean(cdist(lab[mask, 2].reshape((-1, 1)), lab[segments == (i - 1), 2].reshape((-1, 1)), metric='sqeuclidean'))

    # Compute the pairwise distances between features
    color_dist = cdist(color_feat, color_feat, metric='sqeuclidean')
    spatial_dist = cdist(spatial_feat, spatial_feat, metric='euclidean')
    texture_dist = cdist(texture_feat, texture_feat, metric='sqeuclidean')

    # Compute the affinity matrix
    affinity = np.exp(-alpha * (wc * color_dist / sc**2 + ws * spatial_dist / ss**2 + wt * texture_dist / st**2))

    # Convert the affinity matrix to a sparse matrix and save in a txt file
    sparse_affinity = csr_matrix(affinity)
    print(sparse_affinity)
    np.savetxt(r'./affinity/lina.txt', sparse_affinity, fmt='%d', delimiter=',')

    return sparse_affinity

# img = './img/Lena.jpg'
img = io.imread('./img/Lena.jpg')
generate_affinity_matrix(img, 5, alpha=0.1, wc=1, ws=1, wt=1, sc=10, ss=10, st=10)


  (0, 0)	nan
  (0, 1)	nan
  (0, 2)	nan
  (0, 3)	nan
  (0, 4)	nan
  (1, 0)	nan
  (1, 1)	nan
  (1, 2)	nan
  (1, 3)	nan
  (1, 4)	nan
  (2, 0)	nan
  (2, 1)	nan
  (2, 2)	1.0
  (2, 3)	nan
  (2, 4)	nan
  (3, 0)	nan
  (3, 1)	nan
  (3, 2)	nan
  (3, 3)	nan
  (3, 4)	nan
  (4, 0)	nan
  (4, 1)	nan
  (4, 2)	nan
  (4, 3)	nan
  (4, 4)	nan


/Users/liuguangyao/Envs/py37/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/liuguangyao/Envs/py37/lib/python3.9/site-packages/numpy/core/_methods.py:184: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/Users/liuguangyao/Envs/py37/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


ValueError: Expected 1D or 2D array, got 0D array instead

In [ ]:
'''
date: 2023/2/20
author: LIU Guangyao
description: random_walk_with_gcn

'''
import numpy as np
from scipy.sparse import csgraph
from sklearn.preprocessing import normalize
from scipy.sparse import csr_matrix
import networkx as nx
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

def random_walk_with_gcn(affinity_matrix, num_walks, walk_length, num_features):
    # Convert the affinity matrix to a graph representation
    G = nx.from_scipy_sparse_matrix(affinity_matrix)

    # Compute the graph Laplacian
    L = csgraph.laplacian(affinity_matrix, normed=False)

    # Normalize the Laplacian
    L = normalize(L, norm='l1', axis=1)

    # Convert the Laplacian to a sparse tensor
    L = csr_matrix(L)
    indices = np.vstack((L.col, L.row))
    values = L.data
    shape = L.shape
    L = torch.sparse_coo_tensor(indices, values, torch.Size(shape))

    # Create the GCN layers
    gcn1 = GCNConv(num_features, num_features)
    gcn2 = GCNConv(num_features, num_features)

    # Create the random walks
    walks = []
    for i in range(num_walks):
        nodes = np.random.choice(len(G), size=walk_length, replace=True)
        walks.append(nodes)

    # Perform random walk with GCN
    x = torch.eye(len(G), num_features)
    for i in range(num_walks):
        nodes = torch.tensor(walks[i])
        h1 = F.relu(gcn1(x, nodes, L))
        h2 = gcn2(h1, nodes, L)
        x[nodes] = h2

    return x